In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenantid/oauth2/token"}


dbutils.fs.mount(
source = "abfss://landing-zone-2@ecomadlsdarshil.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/ecomdata1",
extra_configs = configs)
  

True

In [0]:
%fs ls '/mnt/ecomdata1/'

path,name,size,modificationTime
dbfs:/mnt/ecomdata1/buyers-raw-2/,buyers-raw-2/,0,1711521400000
dbfs:/mnt/ecomdata1/countries-raw-2/,countries-raw-2/,0,1711521415000
dbfs:/mnt/ecomdata1/sellers-raw-2/,sellers-raw-2/,0,1711521405000
dbfs:/mnt/ecomdata1/users-raw-2/,users-raw-2/,0,1711521393000


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [0]:
spark

In [0]:
# Read parquet file from /mnt/ecomdata1/user-raw-2 folder
userDF = spark.read.format("parquet")\
    .option("header",'true')\
    .option("inferSchema",'true')\
    .load("/mnt/ecomdata1/users-raw-2")

In [0]:
userDF.show(5)

+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierHash|type|   country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasAnyApp|hasAndroidApp|hasIosApp|hasProfilePicture|daysSinceLastLogin|seniority|seniorityAsMonths|seniorityAsYears|countryCode|
+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+-

In [0]:
userDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/users")

In [0]:
buyersDF = spark.read.format("parquet")\
    .option("header",'true')\
    .option("inferSchema",'true')\
    .load("/mnt/ecomdata1/buyers-raw-2")

In [0]:
buyersDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/buyers")

In [0]:
sellersDF = spark.read.format("parquet")\
    .option("header",'true')\
    .option("inferSchema",'true')\
    .load("/mnt/ecomdata1/sellers-raw-2")

sellersDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/sellers")

In [0]:
countriesDF = spark.read.format("parquet")\
    .option("header",'true')\
    .option("inferSchema",'true')\
    .load("/mnt/ecomdata1/countries-raw-2")

countriesDF.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/countries")